## Multi agent Model LINEAR

This script contains only the linearised model of the multi agent system used to model 3 HGVs and a means to collect trajectories

#### 'ThreeVehicles' 

Instantiated with a controller K, and boolean variables defining whether you want system noise or controller noise

#### 'RunEpisode'

Will return lists of states, inputs, costs for a trajectory following the policy defined by K above




In [ ]:
import numpy as np
import pdb
import matplotlib.pyplot as plt
import time
import math


#####################################

class ThreeVehicles():
    
    def __init__(self,K,noisy_model = False, noisy_controller=False):
        
        ###############################################
        #Define all physical constants needed for model
        ###############################################
        
        self.Aa = 20
        self.Cd = 0.6
        self.Cr = 0.005
        self.rho = 1.2
        self.Je = 10
        self.Jw=3
        self.r_w = 0.6
        self.trans_ratio = 1
        self.efficiency = 0.9
        self.m = 30000
        self.g = 9.81
        
        # Create constants to make the model much simpler
        self.K_denominator = self.Jw + (self.m*self.r_w**2) + self.trans_ratio**2 * self.efficiency * self.Je

        self.K1 = self.r_w * self.trans_ratio * self.efficiency /self.K_denominator
        self.K2 = (0.5 * self.Aa * self.rho * self.Cd * self.r_w**2)/self.K_denominator
        self.K3 = (self.Cr*self.m*self.g*self.r_w**2)/self.K_denominator
        self.K4 = (self.m*self.g*self.r_w**2)/self.K_denominator

        # Define coefficients of f_i(d) - the approximation for the non linear equation giving reduction in drag
        self.w1=-0.9379
        self.w2=-0.4502
        self.w3=-0.43735

        self.eps1=12.896
        self.eps2=43.0046
        self.eps3=51.5027


        # Define SAMPLING TIME for discretisation
        # 50ms
        self.Ts = 5e-2

        #########################
        # Define equilibrium point
        #########################
        
        self.v0 = 25
        self.Tau_ref = 0.4  # Time gap between vehicles at steady state
        # d0 MUST REMAIN WITHIN THE RANGE OF THE LINEAR APPROX FOR DRAG REDUCTION (ie less than 15m)
        self.d0 = self.Tau_ref * self.v0 # 25*0.4 = 10
        self.alpha0=0 # slope of road (assume 0 slope for now)
        
        ######################
        #T0 values are: [3875.49 2825.92 2567.12]

        self.T01 = (self.K2 * (1-0.01*(self.w1*self.d0+self.eps1)) * self.v0**2 + self.K3*np.cos(self.alpha0) + self.K4*np.sin(self.alpha0))/self.K1
        self.T02 = (self.K2 * (1-0.01*(self.w2*self.d0+self.eps2)) * self.v0**2 + self.K3*np.cos(self.alpha0) + self.K4*np.sin(self.alpha0))/self.K1
        self.T03 = (self.K2 * (1-0.01*(self.w3*self.d0+self.eps3)) * self.v0**2 + self.K3*np.cos(self.alpha0) + self.K4*np.sin(self.alpha0))/self.K1

        
        ############################
        # Build A and B matrices for linearised model
        ############################

        self.A_v1 = [1-2*self.Ts*self.K2*(1-0.01*(self.w1*self.d0+self.eps1)), self.Ts*self.K2*0.01*self.w1*self.v0**2 , 0 , 0 , 0]
        self.A_d12 = [self.Ts, 1, -self.Ts, 0, 0]
        self.A_v2 = [0, self.Ts*self.K2*0.01*self.w2*self.v0**2 , 1-2*self.Ts*self.K2*(1-0.01*(self.w2*self.d0+self.eps2)), 0 , 0 ]
        self.A_d23 = [0, 0, self.Ts, 1, -self.Ts]
        self.A_v3 = [0, 0, 0, self.Ts*self.K2*0.01*self.w3*self.v0**2 , 1-2*self.Ts*self.K2*(1-0.01*(self.w3*self.d0+self.eps3)) ]

        self.A = np.array([self.A_v1,self.A_d12,self.A_v2,self.A_d23,self.A_v3])
        self.B = 10e+4 * np.array([[self.Ts*self.K1,0,0],[0,0,0],[0,self.Ts*self.K1,0],[0,0,0],[0,0,self.Ts*self.K1]])
        
        ###################
        # Define Cost Matrices
        ###################
        
        #Q1, Q2, Q3 punish v1,v2,v3**2 for each vehicle
        #Q4,Q5 punish the time gap for each vehicle given by: (d12 - Tau_ref * v_following)
        # R1,R2,R3 punish the difference on torque from the reference (HOW to make -ve torque have lower cost)
        self.Q1 = 0.05
        self.Q2 = 0.05
        self.Q3 = 0.05
        self.Q4 = .1
        self.Q5 = .1
        self.R1 = 100
        self.R2 = 100
        self.R3 = 100
        
        self.Q = np.array([[self.Q1, 0, 0, 0, 0],
                           [0, self.Q4, -self.Tau_ref * self.Q4, 0, 0],
                           [0, -self.Tau_ref * self.Q4, self.Q2 + self.Tau_ref**2 * self.Q4, 0, 0],
                           [0,0,0,self.Q5,-self.Tau_ref*self.Q5],
                           [ 0,0,0,-self.Tau_ref * self.Q5, self.Q3 + self.Tau_ref**2 * self.Q5 ]])
        
        
        self.R = np.array([[self.R1,0,0],
                         [0,self.R1,0],
                         [0,0,self.R1]])        
        
        self.disc_fact=0.99
        
        ################
        # Define Gain Matrix (input)
        ################
        
        self.K = K
        
        # st dev of the model noise
        
        if noisy_model == True:
            self.sigma_model = 0.2
        else:
            self.sigma_model = 0
        
        #st dev of the controller noise: u = kx + noise
        self.sigma_controller = 0.25
        self.noisy_controller = noisy_controller
        
    def GetPolicyInput(self,x):
        #For a given state, return the input u according to a defined policy
        # x: 5x1 array
        
        inp = np.matmul(self.K,x).reshape(3,1)
        
        if self.noisy_controller:
            inp += self.GetControllerNoise()
        
        # 3x1 array
        return inp
    
    def GetControllerNoise(self):
        
        contr_noise = self.sigma_controller * np.random.randn(3,1)
        
        return contr_noise
    
    def GetCost(self, x, u):
        #For a given state and input, return the one step cost of this new state
        # x: 5x1 array
        # u: 3x1 array
        
        # x'Ex
        cost1 = np.matmul(np.matmul(x.transpose(),self.Q),x)
        
        # u'Fu
        cost2 = np.matmul(np.matmul(u.transpose(),self.R),u)
        
        
        #scalar
        return (cost1+cost2)[0][0]
    
    def GetNoise(self): #model
        #Returns a vector of noise for the 1st,3rd, and 5th states
        # Noise is necessary for the exploration
        
        w = self.sigma_model * np.array([np.random.randn(1)[0],0,np.random.randn(1)[0],0,np.random.randn(1)[0]]).reshape(5,1)
        
        # 5x1 array
        return w
    
    def GetNextState(self,x,u):
        #x_next = Ax + Bu  (+noise)
        # x: 5x1 array
        # u: 3x1 array
            
        
        x_next_1 = np.matmul(self.A,x)
        x_next_2 = np.matmul(self.B,u)
        x_next_3 = self.GetNoise()
       
        x_next = x_next_1 + x_next_2 + x_next_3
        
        # 5x1 array
        return x_next
    
    def RunEpisode(self,ep_length,state_initial):
        
        #function will carry out an episode of transitions given an initial state vector
        #It will return lists of the states, costs of each transition and inputs
        
        # Format:
        #  ep_length: integer
        #  state_initial: list e.g. [1,2,3,4,5]
        
        
        x = np.array(state_initial).reshape(5,1)
        
        ALL_state_list = [x]
        cost_list = []
        input_list = []
        
        for k in range(ep_length):
            
            u = self.GetPolicyInput(x)
            
            #pdb.set_trace()
            
            input_list.append(u)
            x = self.GetNextState(x,u)
            
            cost = self.GetCost(x,u)
            
            ALL_state_list.append(x)
            cost_list.append(cost)
        
        # ALL_state_list is a list where each element is a 5x1 array
        # cost_list is a list where each element is a scalar
        # input_list is a list where each element is a 3x1 array
        # length of all lists = ep_length
        return ALL_state_list, cost_list, input_list